In [1]:
import json
import re
import pandas as pd
from eval.chat_benchmarks.SimpleQA.judge_prompt import JUDGE_PROMPT
from openai import OpenAI
client = OpenAI()


def read_data(path):
    data = json.load(open(path))
    # data.keys()
    data_key = list(data['results'].keys())[0]
    data_ = data['results'][data_key]['examples']
    df = pd.DataFrame(data_)
    return data, df


In [2]:
path = '/home/dongkeun/ckl_r1/evalchemy/logs/final/nonambigqa_val_1k/confidence_no_trigger/deepseek-ai__DeepSeek-R1-Distill-Qwen-32B/results_2025-04-19T15-40-29.387192.json'
data, df = read_data(path)
df.head(1)

,question,answers,model_think,model_output,model_confidence,grade_letter,correct,not_attempted
0,Who is the author of al capone does my shirts?,['Gennifer Choldenko'],"Okay, so I need to figure out who wrote the bo...",\n\n**Answer**: Gennifer Choldenko\n**Confiden...,Better than even,A,1,0


In [3]:
batch = client.batches.retrieve('batch_680d60f68b808190afa308d3e7693fdc')
file_id = batch.output_file_id
file_response = client.files.content(file_id)
# file_response.text

In [4]:
metadata_path = '_'.join(batch.metadata['description'].split('_')[1:])
assert metadata_path == path, f"Description mismatch: {batch.metadata['description']} != {path}"
mode = batch.metadata['description'].split('_')[0]
print(mode)

uncertainty


In [5]:
records = [json.loads(line) for line in file_response.text.splitlines() if line.strip()]
records_df = pd.DataFrame(records)
records_df.head(1)

,id,custom_id,response,error
0,batch_req_680d66385ad88190a55fee1bc4eb91a2,request-uncertainty_0,"{'status_code': 200, 'request_id': 'c7e03e474c...",None


In [6]:
records_df['custom_id'] = records_df['custom_id'].apply(lambda x: x.split(f'request-{mode}_')[1])
records_df.head(1)

,id,custom_id,response,error
0,batch_req_680d66385ad88190a55fee1bc4eb91a2,0,"{'status_code': 200, 'request_id': 'c7e03e474c...",None


In [7]:
def parse_response(response):
    content = response['body']['choices'][0]['message']['content']
    return content

records_df['model_think'] = records_df['response'].apply(parse_response)
records_df = records_df.drop(columns=['response', 'id', 'error'])
records_df.head(1)

,custom_id,model_think
0,0,"Okay, so I need to figure out who wrote the bo..."


In [8]:
records_df = records_df.set_index('custom_id')
records_df.index.name = None
records_df.head(1)

,model_think
0,"Okay, so I need to figure out who wrote the bo..."


In [9]:
# rename model_think to model_think_original
# df = df.rename(columns={'model_think': 'model_think_original'})
# df.head(1)
records_df = records_df.rename(columns={'model_think': 'prompt'})
records_df.head(1)

,prompt
0,"Okay, so I need to figure out who wrote the bo..."


In [10]:
records_df.index = records_df.index.astype(df.index.dtype)
print(df.index.dtype, df.index[:5])
print(records_df.index.dtype, records_df.index[:5])

int64 RangeIndex(start=0, stop=5, step=1)
int64 Index([0, 1, 2, 3, 4], dtype='int64')


In [11]:
print(f"Length: {len(df)} , {len(records_df)}")

df = df.join(records_df[['prompt']], how='left')
df = df.fillna('NOT_ATTEMPTED')
df.head(1)

Length: 1000 , 1000


,question,answers,model_think,model_output,model_confidence,grade_letter,correct,not_attempted,prompt
0,Who is the author of al capone does my shirts?,['Gennifer Choldenko'],"Okay, so I need to figure out who wrote the bo...",\n\n**Answer**: Gennifer Choldenko\n**Confiden...,Better than even,A,1,0,"Okay, so I need to figure out who wrote the bo..."


In [12]:
# drop model_output, model_confidence, grade_letter, correct, not_attempted
df = df.drop(columns=['model_output', 'model_confidence', 'grade_letter', 'correct', 'not_attempted'])
df.head(1)

,question,answers,model_think,prompt
0,Who is the author of al capone does my shirts?,['Gennifer Choldenko'],"Okay, so I need to figure out who wrote the bo...","Okay, so I need to figure out who wrote the bo..."


In [13]:
from datasets import Dataset

ds = Dataset.from_pandas(df)
ds.push_to_hub(f'nonambigqa_val_1k_{mode}_r1', split='validation')

/home/dongkeun/miniconda3/envs/alchemy-ckl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.34s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/DKYoon/nonambigqa_val_1k_uncertainty_r1/commit/6c919a9c18adfc001aeb6e6a9835025c3fce9aa8', commit_message='Upload dataset', commit_description='', oid='6c919a9c18adfc001aeb6e6a9835025c3fce9aa8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/DKYoon/nonambigqa_val_1k_uncertainty_r1', endpoint='https://huggingface.co', repo_type='dataset', repo_id='DKYoon/nonambigqa_val_1k_uncertainty_r1'), pr_revision=None, pr_num=None)